# Semantic SfM: single scene on synthetic data generated from Kubric 

### 1a. Generate synthetic data
We developed a tool to generate synthetic data using Kubric. Please follow the instructions on the Github repo: https://github.com/ZhiangChen/data_generator_3d

### 1b. Organize data
Create a folder under `semantic_SfM/data` and organize your data following the structures below. 

```
semantic_SfM/data
    ├── kubric_0
        ├── photos
        │       ├── 0.png
        │       ├── 1.png
        │       ├── ...
        │       └── 323.png
        ├── reconstructions
        │       ├── camera_poses.npy
        │       └── combined_point_cloud.las   
        ├── segmentations_gt
        │       ├── 0.npy
        │       ├── 0.png
        │       └── ...
        └── associations
                └── depth
                        ├── 0.npy
                        ├── 0.png
                        └── ...
```

### 2. Segment images using SAM 

#### 2a. SAM segmentation (skipped)

#### 2b. Mask filtering

In [1]:
from ssfm.simple_mask_filter import AreaFilter
area_filter = AreaFilter()

In [2]:
config = {'area_lower_threshold': 100,
'segmentation_folder_path': '../../data/kubric_1/segmentations_gt',
'output_folder_path': '../../data/kubric_1/segmentations_gt_filtered',
'num_processes':8}

area_filter(config)

100%|██████████| 285/285 [00:05<00:00, 52.37it/s]


### 3. Create projection associations

In [1]:
from ssfm.probabilistic_projection import *

In [3]:
import os

scene_dir = '../../data/kubric_0'
pointcloud_path = os.path.join(scene_dir, 'reconstructions', 'combined_point_cloud.las')
associations_folder_path = os.path.join(scene_dir, 'associations')
segmentations_folder_path = os.path.join(scene_dir, 'segmentations_gt_filtered')
photos_folder_path = os.path.join(scene_dir, 'photos')

In [4]:
pointcloud_projector = PointcloudProjection(depth_filtering_threshold=1.8)
pointcloud_projector.read_kubric_camera_parameters(scene_dir)
pointcloud_projector.read_pointcloud(pointcloud_path)

In [5]:
# get image list
photo_folder_path = os.path.join(scene_dir, 'photos')
image_list = [f for f in os.listdir(photo_folder_path) if f.endswith('.png')]
# sort image list based on the number in the file name
image_list.sort(key=lambda x: int(x.split('/')[-1].split('.')[0]))

In [6]:
#pointcloud_projector.parallel_batch_project(image_list, save_folder_path)
pointcloud_projector.parallel_batch_project_joblib(image_list, associations_folder_path, num_workers=16, save_depth=False)

Processing frames: 100%|██████████| 285/285 [00:16<00:00, 17.75it/s]


In [2]:
# build keyimage associations
from ssfm.keyimage_associations_builder import *

In [ ]:
smc_solver = KeyimageAssociationsBuilder(image_list, associations_folder_path, segmentations_folder_path)
smc_solver.build_associations()
smc_solver.build_graph(num_chunks=10)

smc_solver.add_camera_to_graph([scene_dir], camera_type="Kubric")

In [7]:
smc_solver.find_min_cover()

| Metric                                                       | Count      | Percentage           |
----------------------------------------------------------------------------------------------------
| Number of points not covered by any image                    | 677264     | 32.07                |
| Number of points covered by less than or equal to 1 image    | 777788     | 36.83                |
| Number of points covered by less than or equal to 3 images   | 1041895    | 49.34                |
| Number of points covered by less than or equal to 5 images   | 1330600    | 63.01                |


### 5. Estimate memory usage

In [3]:
from ssfm.memory_calculator import memory_calculator

In [12]:
# pointcloud file
las_file = pointcloud_path
# image file sample
image_file = os.path.join(photos_folder_path, image_list[0])
# number of images
num_images = len(image_list)
# number of segmentation ids for each point in the point cloud
num_segmentation_ids = 5

memory_calculator(las_file, image_file, num_images, num_segmentation_ids)

+----------------------------------------+----------------------+
|              Memory Type               | Memory Required (GB) |
+----------------------------------------+----------------------+
|      Segmentation for each image       |    0.00048828125     |
| Pixel2point association for each image |     0.0009765625     |
| Point2pixel association for each image | 0.007866773754358292 |
|                                        |                      |
|      Segmentation for all images       |    0.13916015625     |
| Pixel2point association for all images |     0.2783203125     |
| Point2pixel association for all images |  2.242030519992113   |
|          pc_segmentation_ids           | 0.03933386877179146  |
|         pc_segmentation_probs          | 0.03933386877179146  |
|          keyimage_association          |  0.5605076299980283  |
|                 Total                  |  3.2986863562837243  |
+----------------------------------------+----------------------+


### 6. Run object registration

In [3]:
from ssfm.object_registration import *
from ssfm.post_processing import *

In [4]:
print(pointcloud_path)
print(associations_folder_path)
print(segmentations_folder_path)

../../data/kubric_1/reconstructions/combined_point_cloud.las
../../data/kubric_1/associations
../../data/kubric_1/segmentations_gt_filtered


In [7]:
# Create object registration
obr = ObjectRegistration(pointcloud_path, segmentations_folder_path, associations_folder_path, image_list=image_list, using_graph=True)

# Run object registration
obr.object_registration(iou_threshold=0.50, save_semantics=True)

Processing images: 100%|██████████| 285/285 [02:35<00:00,  1.84it/s]


In [6]:
image_id = 284
semantics_folder_path = os.path.join(associations_folder_path, 'semantics', 'semantics_{}.npy'.format(image_id))
save_las_path = os.path.join(associations_folder_path, 'semantics', 'semantics_{}.las'.format(image_id))
add_semantics_to_pointcloud(pointcloud_path, semantics_folder_path, save_las_path, nearest_interpolation=10)

Before removing small semantics: 
maximum of semantics:  2591
number of unique semantics:  202
After removing small semantics: 
number of unique semantics:  202


In [7]:
print(save_las_path)

# read las 
pc = laspy.read(save_las_path)
# get the semantics from the intensity
semantics = pc.intensity
semantics_ids = np.unique(semantics)
print('number of unique semantics: ', len(semantics_ids))
# print the number of points for each semantics
for i in semantics_ids:
    n = np.sum(semantics == i)
    if n < 100:
        print('semantics id: ', i, ' number of points: ', n)

../../data/kubric_1/associations/semantics/semantics_284.las
number of unique semantics:  201


In [8]:
semantic_pc_file_path = save_las_path
post_processing = PostProcessing(semantic_pc_file_path)
post_processing.shuffle_semantic_ids(exclude_largest_semantic=True)
save_las_path = os.path.join(associations_folder_path, 'semantics', 'semantics_{}_shuffled.las'.format(image_id))
post_processing.save_semantic_pointcloud(save_las_path)

Number of unique semantics:  201


In [9]:
from validation import Validator
validator = Validator(
        save_las_path,
        pointcloud_path,
    )

results = validator.validate(np.arange(0.5, 1.0, 0.05))
mAP = results['AP']
mAR = results['AR']

mAP = np.sum(mAP) / len(mAP)
mAR = np.sum(mAR) / len(mAR)
print('mAP: ', mAP, ' mAR: ', mAR)

Unique semantics in prediction:  201
Unique semantics in ground truth:  201
mAP:  0.8432835820895523  mAR:  0.8432835820895523


In [24]:
mAP, mAR = ([0.9950248756218906,
  0.9900497512437811,
  0.9850746268656716,
  0.9751243781094527,
  0.945273631840796,
  0.8606965174129353,
  0.6716417910447762,
  0.43781094527363185,
  0.263681592039801,
  0.04477611940298507],
 [0.9950248756218906,
  0.9900497512437811,
  0.9850746268656716,
  0.9751243781094527,
  0.945273631840796,
  0.8606965174129353,
  0.6716417910447762,
  0.43781094527363185,
  0.263681592039801,
  0.04477611940298507])